# Dataset [rete]
Registrazione delle misure recuperate dal database x

In [ ]:
setwd(fs::path_abs("~/Local_Workspace/TesiMag"))
library(arrow, warn.conflicts = FALSE)
library(dplyr, warn.conflicts = FALSE)
library(ggplot2, warn.conflicts = FALSE)
library(tsibble, warn.conflicts = FALSE)
library(zeallot, warn.conflicts = FALSE)
library(sf, warn.conflicts = FALSE)
library(tidyr, warn.conflicts = FALSE)
library(assertr, warn.conflicts = FALSE)
options(repr.plot.width = 9, repr.plot.res = 300)

source("notebooks/integrazioni_regionali/nb_tools/state_avail.R")
source("notebooks/integrazioni_regionali/nb_tools/pairing_procedure.R")
source("src/load/load.R")
source("src/analysis/data/clim_availability.R")
source("src/analysis/data/quality_check.R")
source("src/pairing/analysis.R")
source("src/pairing/matching.R")
source("src/pairing/plots.R")
source("src/pairing/combining.R")
source("src/pairing/displaying.R")
source("src/plot/plot_helpers.R")
source("src/database/write.R")
source("src/database/open.R")

first_date <- as.Date("2000-01-01")
last_date <- as.Date("2022-12-31")


In [ ]:
dataset_id <- "x"
state <- "x"


## Caricamento da frammenti

## Metadati
Produco le tabelle dei metadati del dataset e delle stazioni. Controllo con `assertr` e `tsibble` la consistenza delle tabelle.

### Dataset
| dataset_id | entity | reference | 
|------------|--------|-----------|
| utf8()     | utf8() | utf8()    |
| pkey       | obs    | obs       |

### Stazioni
| station_id | station_name | network | state | geometa |additional_meta | dataset_id | original_id |
|------------|--------------|---------|-------|---------|----------------|------------|-------------|
| utf8()     | utf8()       | utf8()  | utf8()|         | utf8()         | utf8()     | utf8()      |
| pkey       | obs          | obs     | obs   |         | fkey           | fkey       | obs         |

I metadati delle stazioni che non rientrano nelle tabelle elencate vanno in una tabella "additional_meta" alla quale puntare con un riferimento

### Serie
| series_id | station_id  | variable | qc_step | merged_from |
|-----------|-------------|----------|---------|-------------|



## Dati
Produco la tabella delle misure. Il formato è row-records:

|series_id|date|value|merged|
|---------|----|-----|------|
|utf8()|date()|float()|bool()|
| fkey    |obs | obs | obs  |

Note:
- `merged` indica se il valore in riga è risultato di un merge. Le caratteristiche del merge sono riportate nella tabella delle stazioni.

Effettuare controllo qualità

In [ ]:
data |>
    as_tsibble(key = series_id, index = date) |>
    gross_errors_check() |>
    repeated_values_check() |>
    verify(qc_gross, qc_repeated)


## Analisi disponibilità dati
Grafici che mostrano la disponibilità di serie mese per mese

In [ ]:
scia_stats <- open.dataset("SCIA", "metadata") |> filter(state == state)
scia_data <- open.dataset("SCIA", "data") |>
    semi_join(scia_stats, by = "identifier") |>
    mutate(db = "SCIA", identifier = cast(identifier, utf8())) |>
    collect()
state_data <- data |>
    left_join(series_table |> select(series_id, station_id, variable), by = "series_id") |>
    collect() |>
    rename(identifier = station_id) |>
    mutate(db = dataset_id) |>
    select(db, identifier, variable, date, value)
state_data |>
    bind_rows(scia_data) |>
    plot_state_avail(first_date, last_date)


In [ ]:
state_boundaries <- load.italian_boundaries("state") |> filter(shapeName == state)
climats_comp <- is_climatology_computable(as_tsibble(state_data, key = c(identifier, variable), index = date), value, .start = first_date, .end = last_date) |>
    left_join(stations_table |> select(station_id, lon, lat) |> collect(), by = join_by(identifier == station_id)) |>
    st_md_to_sf()


## Merging
Gap-filling con altri dataset.
Registrazione dell'esito.